In [1]:
import set_path

import numpy as np
from numba import jit, njit, vectorize, bool_, float64
import numba
import ast
import csv
import time
import pylas
import src.utils.clip_utils as clip_utils

In [2]:
buildings = []
bgt_file = '../datasets/bgt/pand_scrape_119300,485150,119350,485100.csv'

with open(bgt_file, 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        buildings.append(ast.literal_eval(row['pand_polygon']))

In [3]:
las = pylas.read('../datasets/pointcloud/filtered_2386_9702.laz')
xy_points = {'x': las.x, 'y': las.y}

In [4]:
start = time.time()
ray_building_mask = []
for polygon in buildings:
    ray_building_mask.extend(clip_utils.poly_clip(xy_points, polygon, method='ray'))
end = time.time()
print(f'ray_trace finished in {end-start} seconds.')


# warmup
clip_utils.poly_clip(xy_points, buildings[0], method='sm')

start = time.time()
sm_building_mask = []
for polygon in buildings:
    sm_building_mask.extend(clip_utils.poly_clip(xy_points, polygon, method='sm'))
end = time.time()
print(f'sm_parallel finished in {end-start} seconds.')


# warmup
clip_utils.poly_clip(xy_points, buildings[0], method='sm_single')

start = time.time()
sms_building_mask = []
for polygon in buildings:
    sms_building_mask.extend(clip_utils.poly_clip(xy_points, polygon, method='sm_single'))
end = time.time()
print(f'sm_single finished in {end-start} seconds.')

ray_trace finished in 2.0636019706726074 seconds.
sm_parallel finished in 0.2504231929779053 seconds.
sm_single finished in 0.2989022731781006 seconds.


In [9]:
%%timeit
ray_building_mask = []
for polygon in buildings:
    ray_building_mask.extend(clip_utils.poly_clip(xy_points, polygon, method='ray'))

1.82 s ± 5.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit -n10 -r7
sm_building_mask = []
for polygon in buildings:
    sm_building_mask.extend(clip_utils.poly_clip(xy_points, polygon, method='sm'))

237 ms ± 900 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit -n10 -r7
sms_building_mask = []
for polygon in buildings:
    sms_building_mask.extend(clip_utils.poly_clip(xy_points, polygon, method='sm_single'))

287 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
ray_building_mask == sm_building_mask

True

In [6]:
sms_building_mask == ray_building_mask

True